## Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
import torch

In [ ]:
# https://www.kaggle.com/datasets/benhamner/sf-bay-area-bike-share?select=status.csv
dataset_station_statut = pd.read_csv("../../Bike_Data/status.csv")
dataset_station = pd.read_csv("../../Bike_Data/station.csv")

## Class TimeSeriesDataset

In [ ]:
# Define a PyTorch dataset to generate input/target pairs for the LSTM model
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size, stride):
        self.data = data
        self.window_size = window_size
        self.stride = stride

    def __len__(self):
        return len(self.data) - self.window_size

    def __getitem__(self, idx):
        inputs = self.data[idx:idx+self.window_size]
        target = self.data[idx+self.window_size]
        return inputs, target

## Class LSTM v1

In [ ]:
# Define your LSTM model here with num_layers LSTM layers and 1 fully connected layer
class LSTMModel_v1(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = torch.nn.LSTM(input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc = torch.nn.Linear(self.hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

## Station statut dataset

## Station dataset

In [ ]:
dataset_station.head(3)

In [ ]:
dataset_station.shape

## Merge dataset Station statut and Station

In [ ]:
dataset = pd.merge(dataset_station, dataset_station_statut, left_on='id', right_on='station_id')

In [ ]:
dataset.head(3)

In [ ]:
dataset.shape

## 1/ (Mulivariate) Selection three station to make prediction

### All id station available

In [ ]:
print(dataset['id'].unique())
print(len(dataset['id'].unique()))

In [ ]:
dataset_station_id = dataset.loc[dataset['id'].isin([42, 70, 60])]

In [ ]:
dock_count = dataset_station_id['dock_count'].unique()
print(dock_count)

In [ ]:
dataset_station_id.shape

In [ ]:
dataset_station_id.head(3)

In [ ]:
dataset_station_id.tail(5)

### Drop columns

In [ ]:
dataset_station_id_transform = dataset_station_id.drop(["name", 'lat', 'long', 'id', 'city', 'installation_date', "docks_available", "dock_count"], axis=1)

In [ ]:
dataset_station_id_transform.head(3)

### Check presence of null and NaN values

In [ ]:
dataset_station_id_transform.isna().sum()

In [ ]:
dataset_station_id_transform[dataset_station_id_transform.isna().any(axis=1)]

In [ ]:
dataset_station_id_transform.dtypes

### Conversion column time to datetime

In [ ]:
dataset_station_id_transform['time'] = pd.to_datetime(dataset_station_id_transform['time'], format="mixed")

In [ ]:
dataset_station_id_transform.dtypes

In [ ]:
dataset_station_id_transform.head(10)

In [ ]:
dataset_station_id_transform.tail(10)

### Train Function

In [ ]:
def plot_loss_valid(valid_losses):
    plt.plot(valid_losses, label = 'Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Validation Loss')
    plt.show()

In [ ]:
# Train your model and evaluate on the validation set
def train_model(model, optimizer, criterion, train_loader, valid_loader, num_epochs):
    num_epochs = num_epochs
    best_val_loss = float('inf')
    train_losses = []
    valid_losses = []
    for epoch in range(num_epochs):
        train_loss = 0.0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs, targets.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_losses.append(loss.item())
        val_loss = 0.0
    
        for inputs, targets in valid_loader:
            outputs = model(inputs.float())
            loss = criterion(outputs, targets.float())
            val_loss += loss.item()
        
        val_loss /= len(valid_loader)
        valid_losses.append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model_LSTM.pth')
    
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        
    plot_loss_valid(valid_losses)
    
    

### Test Function

In [ ]:
def test_model(best_model, test_loader, criterion):
    # Load the best model and evaluate on the test set
    best_model.double()
    best_model.eval()

    # Evaluate the model on the test set
    test_loss = 0.0
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            x = torch.Tensor(inputs).unsqueeze(1).to(device)
            y = torch.Tensor(targets).unsqueeze(0).to(device)
            outputs = best_model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            # Save the predictions and actual values for plotting later
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())
    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}")
    # Concatenate the predictions and actuals
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)

    return (predictions, actuals)

### Print Metrics for each month Function

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

def result_prediction_by_month(predictions, actuals):
    indices_by_month = []
    EPSILON = 1e-10
    for i in range(1): 
        grouped_data = test_data.groupby(pd.Grouper(freq='M'))
        for name , group in grouped_data:
            indices = np.where(test_data.index.isin(group.index))[0]
            indices_by_month.append((name.strftime('%B'), indices))
            
        for name, indice in indices_by_month :
            y_pred = predictions[indice-window_size,i]
            y_true = actuals[indice-window_size,i]
            
            mae = mean_absolute_error(y_true, y_pred)
            mape = mean_absolute_percentage_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            maape =  np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + EPSILON))))
            # Add evaluation metrics to the plot
            print(f'\n{name}')
            print(f'MAE: {mae:.2f}')
            print(f'MAPE: {mape:.2f}')
            print(f'RMSE: {rmse:.2f}')
            print(f'MAAPE: {maape:.2f}')

### Print Plot to see accuracy and some Metrics Function

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

def plot_result_prediction(predictions, actuals):
    EPSILON = 1e-10
    for i in range(3): 
        for y in range(0, len(test_data), 8):
            plt.figure(figsize=(28, 5))
            plt.rcParams.update({'font.size': 16})  # Augmenter la taille de la police
            plt.gcf().set_size_inches(20, 8)  # Augmenter la taille de la figure

            debut = y
            fin = min(y + 8, len(test_data))
                
            plt.title('Actual vs Prediction of station N° {}'.format(pivoted_df_station_id.columns[i]))
            y_pred = predictions[debut:fin,i]
            y_true = actuals[debut:fin,i]
            plt.plot(test_data.index[debut + window_size: fin + window_size],predictions[debut:fin, i], label='Actuals')
            plt.plot(test_data.index[debut + window_size: fin + window_size],actuals[debut:fin, i], label='Predictions')
            mae = mean_absolute_error(y_true, y_pred)
            mape = mean_absolute_percentage_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            maape =  np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + EPSILON))))
            # Add evaluation metrics to the plot
            plt.annotate(f'MAE: {mae:.2f}', xy=(0.005, 0.85), xycoords='axes fraction')
            plt.annotate(f'MAPE: {mape:.2f}', xy=(0.005, 0.80), xycoords='axes fraction')
            plt.annotate(f'RMSE: {rmse:.2f}', xy=(0.005, 0.75), xycoords='axes fraction')
            plt.annotate(f'MAAPE: {maape:.2f}', xy=(0.005, 0.70), xycoords='axes fraction')

            # Set x and y labels
            plt.xlabel('Time')
            plt.ylabel('Value')
            plt.legend()
            print(mae,mape,rmse,maape)
            plt.show()

### First Experimentation group by day and hour 

#### Trying to forecast the availability of bikes 

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
pivoted_df_station_id = dataset_station_id_transform.pivot_table(index='time', columns='station_id', values='bikes_available')

In [ ]:
df_weekday_hour = pivoted_df_station_id.groupby(pd.Grouper(freq='H'), dropna=True).mean()

In [ ]:
len(df_weekday_hour[df_weekday_hour.isna().any(axis=1)])

In [ ]:
df_weekday_hour = df_weekday_hour.dropna()

In [ ]:
len(df_weekday_hour[df_weekday_hour.isna().any(axis=1)])

In [ ]:
df_weekday_hour_prep = df_weekday_hour.copy()

In [ ]:
df_weekday_hour_prep.head(10)

In [ ]:
df_weekday_hour_prep[df_weekday_hour_prep.isna().any(axis=1)]

In [ ]:
train_data = df_weekday_hour_prep[:'2014-10-31 12:00:00']
valid_data = df_weekday_hour_prep['2014-10-31 12:00:00':'2015-02-01 12:00:00']
test_data = df_weekday_hour_prep['2015-02-01 12:00:00':'2015-08-31 12:00:00']

In [ ]:
# Define the sliding window size and stride
window_size = 7
stride = 1
layers = 3
hidden_size = 32
input = 3
output = 3
# Create datasets and data loaders for training, validation, and test sets
train_dataset = TimeSeriesDataset(train_data.values, window_size, stride)
valid_dataset = TimeSeriesDataset(valid_data.values, window_size, stride)
test_dataset = TimeSeriesDataset(test_data.values, window_size, stride)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Instantiate your LSTM model and define the loss function and optimizer
model = LSTMModel_v1(input_size=input, hidden_size=hidden_size, num_layers=layers, output_size=output)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, optimizer, criterion, train_loader, valid_loader, 200)

In [ ]:
best_model =  LSTMModel_v1(input_size=input, hidden_size=32, num_layers=layers, output_size=output)
best_model.load_state_dict(torch.load('best_model_LSTM.pth'.format(input)))
predictions, actuals = test_model(best_model, test_loader, criterion)

In [ ]:
result_prediction_by_month(predictions, actuals)

In [ ]:
plot_result_prediction(predictions, actuals)

### Second Experimentation group by day and 10 min

In [ ]:
df_ten_minutes = dataset_station_id_transform.copy()

In [ ]:
df_ten_minutes = df_ten_minutes.groupby(by=pd.Grouper(key='time', freq='10min'), dropna=True).mean()

In [ ]:
df_ten_minutes.head(5)

In [ ]:
len(df_ten_minutes[df_ten_minutes.isna().any(axis=1)])

In [ ]:
df_ten_minutes = df_ten_minutes.dropna()

In [ ]:
len(df_ten_minutes[df_ten_minutes.isna().any(axis=1)])

In [ ]:
df_ten_minutes_prep = df_ten_minutes.drop(['dock_count', "docks_available"], axis=1)

In [ ]:
train_data = df_ten_minutes_prep[:'2014-10-31 12:00:00']
val_data = df_ten_minutes_prep['2014-10-31 12:00:00':'2015-02-01 12:00:00']
test_data = df_ten_minutes_prep['2015-02-01 12:00:00':'2015-08-31 12:00:00']

In [ ]:
# Define the sliding window size and stride
window_size = 7
stride = 1
layers = 3
hidden_size = 32
n = 1
# Create datasets and data loaders for training, validation, and test sets
train_dataset = TimeSeriesDataset(train_data.values, window_size, stride)
val_dataset = TimeSeriesDataset(val_data.values, window_size, stride)
test_dataset = TimeSeriesDataset(test_data.values, window_size, stride)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Instantiate your LSTM model and define the loss function and optimizer
model = LSTMModel_v1(input_size=n, hidden_size=hidden_size, num_layers=layers, output_size=n)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, optimizer, criterion, train_loader, valid_loader, 100)

In [ ]:
best_model = LSTMModel_v1(input_size=n, hidden_size=32, num_layers=layers, output_size=n)
best_model.load_state_dict(torch.load('best_model_LSTM.pth'.format(n)))
predictions, actuals = test_model(best_model, test_loader, criterion)

In [ ]:
result_prediction_by_month(predictions, actuals)

In [ ]:
plot_result_prediction(predictions, actuals)

### Third Experimentation group by day and 30 min

In [ ]:
df_thirty_minutes = dataset_station_id_transform.copy()

In [ ]:
df_thirty_minutes = df_thirty_minutes.groupby(by=pd.Grouper(key='time', freq='30min'), dropna=True).mean()

In [ ]:
df_thirty_minutes.head(5)

In [ ]:
len(df_ten_minutes[df_ten_minutes.isna().any(axis=1)])

In [ ]:
df_thirty_minutes = df_thirty_minutes.dropna()

In [ ]:
df_thirty_minutes_prep = df_ten_minutes.drop(['dock_count', "docks_available"], axis=1)

In [ ]:
train_data = df_thirty_minutes_prep[:'2014-10-31 12:00:00']
val_data = df_thirty_minutes_prep['2014-10-31 12:00:00':'2015-02-01 12:00:00']
test_data = df_thirty_minutes_prep['2015-02-01 12:00:00':'2015-08-31 12:00:00']

In [ ]:
# Define the sliding window size and stride
window_size = 7
stride = 1
layers = 3
hidden_size = 32
n = 1
# Create datasets and data loaders for training, validation, and test sets
train_dataset = TimeSeriesDataset(train_data.values, window_size, stride)
val_dataset = TimeSeriesDataset(val_data.values, window_size, stride)
test_dataset = TimeSeriesDataset(test_data.values, window_size, stride)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Instantiate your LSTM model and define the loss function and optimizer
model = LSTMModel_v1(input_size=n, hidden_size=hidden_size, num_layers=layers, output_size=n)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, optimizer, criterion, train_loader, valid_loader, 100)

In [ ]:
best_model = LSTMModel_v1(input_size=n, hidden_size=32, num_layers=layers, output_size=n)
best_model.load_state_dict(torch.load('best_model_LSTM.pth'.format(n)))
predictions, actuals = test_model(best_model, test_loader, criterion)

In [ ]:
result_prediction_by_month(predictions, actuals)

In [ ]:
plot_result_prediction(predictions, actuals)